In [39]:
import numpy as np

def my_matmult(m1, m2):
    ans_mat = np.empty((0,m2.shape[1]))
    for row in m1:
        ans_row = np.array([])
        for col in m2.T:
            ans_row = np.append(ans_row, np.dot(row, col))
        ans_mat = np.vstack((ans_mat,ans_row))
    return ans_mat

a = np.random.rand(3, 2)
b = np.random.rand(2, 3)

print(np.array_equal(a@b, my_matmult(a,b)))


True


In [40]:
def tensor_product(m1, m2):
  ans = np.empty( (0, m1.shape[-2], m2.shape[-1]) )
  for i in range(m1.shape[0]):
    matmult = np.expand_dims(my_matmult(m1[i], m2[i]), axis=0)
    ans = np.vstack((ans, matmult))
  return ans

a = np.random.rand(2, 3, 4)
b = np.random.rand(2, 4, 2)

print(np.allclose(tensor_product(a,b), a @ b))


True


In [41]:
def nd_tensor_product(m1, m2):
  ans_shape = m1.shape[:-2] + (m1.shape[-2], m2.shape[-1])
  ans = np.empty(ans_shape)
  for i in range(m1.shape[0]):
    if len(m1.shape) == 3:
        ans[i] = my_matmult(m1[i], m2[i])
    else:
        ans[i] = nd_tensor_product(m1[i], m2[i])
  return ans

a = np.random.rand(3, 2, 3, 4)
b = np.random.rand(3, 2, 4, 2)

print(np.allclose(nd_tensor_product(a,b), a @ b))

True
